# Solver for a multifocal Koehler integrator

In [1]:
import numpy as np
from scipy.optimize import minimize

from kmdouglass.udesigner import Units
from kmdouglass.udesigner.mfki import compute_results, DEFAULTS

## Define the input search space

In this case, we'll look at a range of (microlens array) MLA focal lengths and pitches.

In [6]:
# Minimum and maximum values for the variables
mla_f = np.array([4.88, 10.0])
mla_pitch = np.array([300.0, 600.0])

# Choose a random set of initial values
mla_f0 = np.random.uniform(*mla_f)
mla_pitch0 = np.random.uniform(*mla_pitch)

x0 = np.array([mla_f0, mla_pitch0])

## Define the cost function

The cost function is defined based on this intuition:

1. We should be as close to the target field size as possible
2. The spot size should be as small as possible
3. The Fresnel number should be as large as possible
4. The homogeneity factor should be as large as possible

The objective function used is:

$O = \underset{f, p}{\operatorname{argmin}} \left( a |S_{sample} \left( f, p \right) - S_{target}|^2 + b r_{sample} - c \text{FN} - d B\right)$

where:

- $f$: MLA focal length
- $p$: MLA pitch
- $S_{sample}$: The flat field size on the sample
- $S_{target}$: The target flat field size
- $r_{sample}$: The spot size on the sample
- $\text{FN}$: the Fresneal number
- $B$: The homogeneity factor
- $a$, $b$, $c$, $d$: hyperparameters for each factor

The optimizer finds the values of $f$ and $p$ that minimize this function.

In [7]:
def cost_function(
        mla_f: float,
        mla_pitch: float,
        mla_f_units: Units = Units.mm,
        mla_pitch_units: Units = Units.um,
        field_size_target: float = 160e-6,
        field_size_hyperparam: float = 10.0,
        spot_size_hyperparam: float = 1.0,
        fresnel_number_hyperparam: float = 1.0,
        homogeneity_hyperparam: float = 1.0,
) -> float:
    inputs = DEFAULTS.copy()
    inputs["mla.focal_length"] = mla_f
    inputs["mla.focal_length.units"] = mla_f_units
    inputs["mla.pitch"] = mla_pitch
    inputs["mla.pitch.units"] = mla_pitch_units

    results = compute_results(inputs)

    flat_field_size_sample_plane = results["flat_field_size_sample_plane"]["value"] * results["flat_field_size_sample_plane"]["units"].value
    excitation_spot_size_sample_plane = results["excitation_spot_size_sample_plane"]["value"] * results["excitation_spot_size_sample_plane"]["units"].value
    fresnel_number = results["fresnel_number"]["value"]
    homogeneity = results["homogeneity"]["value"]

    size_err = np.abs(flat_field_size_sample_plane - field_size_target)**2

    cost = (
        field_size_hyperparam * size_err
        + spot_size_hyperparam * excitation_spot_size_sample_plane
        - fresnel_number_hyperparam * fresnel_number
        - homogeneity_hyperparam * homogeneity
    )

    return cost


def objective(params):
    mla_f, mla_pitch = params
    return cost_function(mla_f, mla_pitch)

## Run the optimization routine

In [8]:
result = minimize(
    objective,
    x0,
    bounds=[(mla_f.min(), mla_f.max()), (mla_pitch.min(), mla_pitch.max())],
    method="Nelder-Mead",
)

In [9]:
result

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: -49.492393412087395
             x: [ 4.880e+00  6.000e+02]
           nit: 6
          nfev: 12
 final_simplex: (array([[ 4.880e+00,  6.000e+02],
                       [ 4.880e+00,  6.000e+02],
                       [ 4.880e+00,  6.000e+02]]), array([-4.949e+01, -4.949e+01, -4.949e+01]))